In [2]:
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
from fiona.env import Env
from shapely.ops import unary_union
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

DATA_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\final"
os.makedirs(OUT_DIR, exist_ok=True)

WGS84, UTM51 = "+proj=longlat +datum=WGS84 +no_defs", "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VMIN, VMAX = 0, 100
CMAP = mpl.colormaps["RdBu"]
HALO_RADIUS_M, HALO_ALPHA_MAX, HALO_SIGMA_FRAC = 800, 0.35, 2.5
COVERAGE_MAX_DIST_M = 3000
VARIOGRAM_MODEL, RETRY_NUGGET = "spherical", 1e-3

target_date = "2025-02-06"   # <<< change this date
f = os.path.join(DATA_DIR, f"date_{target_date}.gpkg")
if not os.path.exists(f): raise SystemExit(f"Not found: {f}")

with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

g = gpd.read_file(f)
if g.crs is None: g = g.set_crs(WGS84)
if "geometry" not in g or g.geometry.is_empty.all():
    lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
    lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
    g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
pm_col = next((c for c in g.columns if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5")), None)
pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
pts["x"]=pts.geometry.x; pts["y"]=pts.geometry.y
pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))
x = pts["x"].values; y = pts["y"].values; z = pts[pm_col].astype(float).clip(lower=0).values

minx,miny,maxx,maxy = geom.bounds
xs = np.arange(minx, maxx+GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy+GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

try:
    OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL, enable_plotting=False, verbose=False, coordinates_type="euclidean")
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)
except LinAlgError:
    OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL, nugget=RETRY_NUGGET, enable_plotting=False, verbose=False, coordinates_type="euclidean")
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)

# masks
tree = cKDTree(np.c_[x,y]); dists,_ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)
centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
inside  = centers.within(geom).values.reshape(zg.shape)
zmask = np.where(cover & inside, np.maximum(zg,0.0), np.nan)

# plot (halo on peak)
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
im = ax.imshow(zmask, extent=[xs[0], xs[-1], ys[0], ys[-1]], origin="lower", cmap=CMAP, vmin=VMIN, vmax=VMAX)
pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)

i_max = int(np.nanargmax(z)); cx, cy = float(x[i_max]), float(y[i_max])
radius = HALO_RADIUS_M; sigma = radius / HALO_SIGMA_FRAC
step = max(20, GRID_RES_M//2); n = int(2*radius/step)+1
hx = np.linspace(cx-radius, cx+radius, n); hy = np.linspace(cy-radius, cy+radius, n)
HX, HY = np.meshgrid(hx, hy); R2 = (HX-cx)**2 + (HY-cy)**2
alpha = np.exp(-R2/(2*sigma**2)); alpha[R2>radius**2]=0
halo = np.zeros((n,n,4)); halo[...,0]=0.85; halo[...,1]=0.15; halo[...,2]=0.10; halo[...,3]=0.35*alpha
ax.imshow(halo, extent=[cx-radius, cx+radius, cy-radius, cy+radius], origin="lower", interpolation="bilinear", zorder=3)
ax.scatter([cx],[cy], marker="*", s=260, facecolor="white", edgecolor="black", linewidths=1.0, zorder=4)
ax.text(cx, cy, f"  max {z[i_max]:.1f} µg/m³", fontsize=10, weight="bold", color="black")

cb = plt.colorbar(im, ax=ax, shrink=0.8); cb.set_label("Estimated PM₂.₅ (µg/m³)")
ax.set_title(f"Estimated PM₂.₅ • {target_date}")
ax.set_axis_off()
out = os.path.join(OUT_DIR, f"final_policy_{target_date}.png")
plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
print("Saved:", out)

Saved: C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\final\final_policy_2025-02-06.png
